# Copulas Example

This notebook demonstrates the use of copulas in the Proteus Actuarial Library to model dependencies between different lines of business in insurance.

## Setup and Configuration

Import necessary libraries and set simulation parameters.

In [1]:
# Enable plotting in Jupyter notebooks
# The development container sets PAL_SUPPRESS_PLOTS=true by default to prevent
# plots from appearing during automated testing and CI/CD runs. However, we want
# to see plots when running notebooks interactively, so we override it here.
import os

os.environ["PAL_SUPPRESS_PLOTS"] = "false"

In [ ]:
import plotly.graph_objects as go
from pal import config, copulas, distributions
from pal.frequency_severity import FrequencySeverityModel
from pal.stats import tvar
from pal.variables import ProteusVariable

config.n_sims = 100000

In [3]:
config.n_sims = 100000

## Define Lines of Business

Set up the lines of business (LOBs) for our insurance portfolio.

In [4]:
lobs = ["Motor", "Property", "Liability", "Marine", "Aviation"]

## Generate Individual Large Losses

Create frequency-severity models for large losses in each line of business using:
- Poisson distribution for frequency (mean=5)
- Generalized Pareto Distribution (GPD) for severity

In [5]:
# Generate the individual large losses by class
individual_large_losses_by_lob = ProteusVariable(
    dim_name="class",
    values={
        name: FrequencySeverityModel(
            distributions.Poisson(mean=5),
            distributions.GPD(shape=0.33, scale=100000, loc=1000000),
        ).generate()
        for name in lobs
    },
)

## Generate Attritional Losses

Model attritional (small, frequent) losses using Gamma distributions with varying parameters for each LOB.

In [6]:
# Generate the attritional losses by class
attritional_losses_by_lob = ProteusVariable(
    "class",
    values={
        lob: distributions.Gamma(alpha=i + 1, theta=1000000).generate()
        for i, lob in enumerate(lobs)
    },
)

## Apply Loss Adjustment Expenses (LAE)

Add a 5% LAE factor to large losses.

In [7]:
large_losses_with_lae = individual_large_losses_by_lob * 1.05

## Aggregate Large Losses by Class

In [ ]:
# create the aggregate losses by class

aggregate_large_losses_by_class = ProteusVariable(
    "class",
    {name: large_losses_with_lae[name].aggregate() for name in lobs},
)

## Apply Pairwise Copulas

Correlate attritional and large losses within each LOB using Gumbel copulas.

In [ ]:
# correlate the attritional and large losses. Use a pairwise copula to do this
for lob in lobs:
    copulas.GumbelCopula(theta=1.2, n=2).apply(
        [
            aggregate_large_losses_by_class[lob],
            attritional_losses_by_lob[lob],
        ]
    )

## Calculate Total Losses by LOB

In [10]:
# calculate the total losses
total_losses_by_lob = aggregate_large_losses_by_class + attritional_losses_by_lob

## Apply Multi-Dimensional Copula

Model dependencies between different LOBs using a Student's t-copula with a correlation matrix.

In [11]:
# apply a copula to the total losses by lob
correlation_matrix = [
    [1.0, 0.5, 0.3, 0.2, 0.1],
    [0.5, 1.0, 0.4, 0.3, 0.2],
    [0.3, 0.4, 1.0, 0.5, 0.4],
    [0.2, 0.3, 0.5, 1.0, 0.6],
    [0.1, 0.2, 0.4, 0.6, 1.0],
]
copulas.StudentsTCopula(correlation_matrix, 5, "linear").apply(total_losses_by_lob)

## Apply Stochastic Inflation

Model inflation as a stochastic process with normal distribution (mean=5%, std=2%).

In [ ]:
# apply stochastic inflation
stochastic_inflation = distributions.Normal(0.05, 0.02).generate()
inflated_total_losses_by_lob = total_losses_by_lob * (1 + stochastic_inflation)

## Calculate Portfolio Total Losses

In [ ]:
# create the total losses
total_inflated_losses = sum(inflated_total_losses_by_lob)

## Risk Metrics

Calculate Tail Value at Risk (TVaR) at the 99th percentile.

In [14]:
print(f"TVaR(99%): {tvar(total_inflated_losses, 99):,.2f}")

TVaR(99%): 96,498,973.47


## Visualize CDF

Display the cumulative distribution function of total inflated losses.

In [ ]:
total_inflated_losses.show_cdf()

## Visualize Dependency Structure

Create a scatter plot showing the rank correlation between Motor and Property losses.

In [ ]:
fig = go.Figure(
    go.Scattergl(
        x=inflated_total_losses_by_lob["Motor"].ranks.values,
        y=inflated_total_losses_by_lob["Property"].ranks.values,
        mode="markers",
    ),
    layout={
        "xaxis": {"title": "Motor - Rank"},
        "yaxis": {"title": "Property - Rank"},
        "title": "Scatter plot of Motor and Property losses",
    },
)
fig.show()  # type: ignore[reportUnknownMemberType]